# Experimentovanie a vyhodnocovanie

Miroslav Čulík a Andrej Gáfrik

TODOs:
- prekopčiť veci z minulých správ do finálnej správy na overleafe https://www.overleaf.com/project/5e7101e344e1ac0001e3041b
- Experimenty
    - DM metóda: experimentovanie, nastavenie parametrov, trénovanie, testovanie, ...
    - vyhodnotenie: vyhodnotenie výsledkov, porovnanie metód (aj s publikovanými prácami)
- Zhrnutie
    - zhodnotenie toho, čo sa v rámci projektu podarilo
    - náčrt ďalšieho vhodného smerovania projektu
- Literatúra: zoznam použitej literatúry a ich citovanie v správe

- pridat vizualizacie k Decision Tree

- pridat nove modely
    - RF
    - SVR

- hyperparameter tuning
    - manual pri RF
    - gridsearch
    - random search
    

In [1]:
cd ..

C:\Users\Mirec\Desktop\08_fiit_ls_2019_2020\OZNAL\projekt\oznal_project


In [26]:
import seaborn as sns
import random
import itertools
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.special import inv_boxcox
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from numpy.polynomial.polynomial import polyfit

from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import SGDRegressor

from src import analysis, preprocessing2, feature_selection2, metrics2

pd.set_option('max_rows', None)
pd.set_option('max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
path_housing = "data\\kc_house_data.csv"
df = pd.read_csv(path_housing, sep=",")
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.00000,3,1.00000,1180,5650,1.00000,0,0,3,7,1180,0,1955,0,98178,47.51120,-122.25700,1340,5650
1,6414100192,20141209T000000,538000.00000,3,2.25000,2570,7242,2.00000,0,0,3,7,2170,400,1951,1991,98125,47.72100,-122.31900,1690,7639
2,5631500400,20150225T000000,180000.00000,2,1.00000,770,10000,1.00000,0,0,3,6,770,0,1933,0,98028,47.73790,-122.23300,2720,8062
3,2487200875,20141209T000000,604000.00000,4,3.00000,1960,5000,1.00000,0,0,5,7,1050,910,1965,0,98136,47.52080,-122.39300,1360,5000
4,1954400510,20150218T000000,510000.00000,3,2.00000,1680,8080,1.00000,0,0,3,8,1680,0,1987,0,98074,47.61680,-122.04500,1800,7503


In [4]:
X_train, X_valid, X_test, y_train, y_valid, y_test, price_lambda = preprocessing2.run_pipeline(df)

In [5]:
X_train.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode_1,zipcode_2,zipcode_3,zipcode_4,zipcode_5,zipcode_6,zipcode_7,lat,long,sqft_living15,sqft_lot15,recon_age,price_per_sqft
12496,1982201345,2,1.00000,7.38084,4.19603,1.00000,0,0,4,7,3.41952,0.00000,1944,0,0,0,0,0,0,0,1,47.66390,-122.36400,7.04752,3.99849,70,0.00248
775,4040800810,3,2.25000,8.50943,4.29079,1.00000,0,0,4,8,3.46649,31.62278,1963,0,0,0,0,0,0,1,0,47.61880,-122.11400,7.63530,4.10521,51,0.00172
17400,7525410190,3,1.75000,8.98009,4.52123,1.50000,0,0,3,8,3.66089,0.00000,1979,0,0,0,0,0,0,1,1,47.57470,-122.03500,7.85941,4.09276,35,0.00131
17038,2326059099,4,2.50000,9.14293,4.54771,2.00000,0,0,3,9,3.68127,0.00000,2001,0,0,0,0,0,1,0,0,47.72320,-122.13100,8.11672,4.09276,13,0.00157
5043,4219400290,5,2.75000,8.98009,4.32003,1.50000,0,0,3,8,3.66089,0.00000,1939,0,0,0,0,0,1,0,1,47.65520,-122.27800,7.98616,4.06938,75,0.00169


## Experimenty

Po predspracovaní dát nami vytvorenou pipeline, sme mali dáta rozdelené v pomere 70:20:10 (trénovacia : validačná : testovacia množina). Použili sme implementácie algoritmov z modulu sklearn. Najlepšie výsledky podľa metriky R$^2$ dosiahol model polynomiálnej regresie 3. stupňa s použitím selekcie čŕt RFE (0.916 na trénovacej a 0.828 na testovacej množine), druhý najlepší výsledok podľa tejto metriky dosiahol model regresného rozhodovacieho stromu(0.861 na trénovacej a 0.802 na testovacej množine). V prípade modelu jednoduchej lineárnej regresie sme vyskúšali aj 5, resp. 10 - násobnú krížovú validáciu, na porovnanie výsledkov s modelom jednoduchej lineárnej regresie trénovaného tradičným spôsobom, avšak tento prístup nám nepreukázal žiaden signifikantný rozdiel vo výsledkoch pri modeloch jednoduchej lineárnej regresie.
 

In [153]:
df_results['params'][0]

{'C': 1.7214568800839372,
 'loss': 'epsilon_insensitive',
 'max_iter': 20000,
 'tol': 0.0004935410206914095}

### Jednoduchá lineárna regresia - sqft_living

In [6]:
reg=linear_model.LinearRegression()
x_train=np.array(X_train['sqft_living']).reshape(-1,1)
y_train=np.array(y_train).reshape(-1,1)
reg.fit(x_train,y_train)

x_test=np.array(X_valid['sqft_living']).reshape(-1,1)
y_test=np.array(y_valid).reshape(-1,1)
pred=reg.predict(x_test)
print('linear model - sqft_living')

metrics2.evaluate(reg, x_train, y_train, x_test, y_test, pred, price_lambda)

linear model - sqft_living
Mean Squared Error 60,379,753,642.67178
Root Mean Squared Error 245,722.92047
Root Mean Squared Log Error 0.39115
R squared training 0.451
R squared testing 0.402
Adjusted-R squared training 0.451
Adjusted-R squared testing 0.402
intercept [3.78442207]
coefficient [[0.02965959]]


#### k-násobná krížová validácia pri jednoduchej lineárnej regresii

In [7]:
reg=linear_model.LinearRegression()
k = 5

X_try = pd.concat([X_train['sqft_living'], X_valid['sqft_living']])
y_try = np.concatenate((np.array(y_train).reshape(-1,1), np.array(y_valid).reshape(-1,1)), axis=0)

scores = cross_val_score(reg,  np.array(X_try).reshape(-1,1), np.array(y_try).reshape(-1,1), cv=k)
print("Cross-validated scores:", scores.mean())
k = 10
scores = cross_val_score(reg,  np.array(X_try).reshape(-1,1), np.array(y_try).reshape(-1,1), cv=k)
print("Cross-validated scores:", scores.mean())

Cross-validated scores: 0.4388515347655395
Cross-validated scores: 0.4385100197794961


### Jednoduchá lineárna regresia - grade

In [8]:
reg=linear_model.LinearRegression()
x_train=np.array(X_train['grade']).reshape(-1,1)
y_train=np.array(y_train).reshape(-1,1)
reg.fit(x_train,y_train)

x_test=np.array(X_valid['grade']).reshape(-1,1)
y_test=np.array(y_valid).reshape(-1,1)
pred=reg.predict(x_test)
print('linear model - grade')

metrics2.evaluate(reg, x_train, y_train, x_test, y_test, pred, price_lambda)

linear model - grade
Mean Squared Error 55,001,939,903.94096
Root Mean Squared Error 234,524.92384
Root Mean Squared Log Error 0.38054
R squared training 0.477
R squared testing 0.432
Adjusted-R squared training 0.477
Adjusted-R squared testing 0.432
intercept [3.93205218]
coefficient [[0.01344416]]


### Viacnásobná lineárna regresia - part1

In [9]:
features1 = feature_selection2.feature_filter(X_train, y_train, threshold=0.5)
reg=linear_model.LinearRegression()
reg.fit(X_train[features1],y_train)

pred=reg.predict(X_valid[features1])

print('multiple linear regression 1')
metrics2.evaluate(reg, X_train[features1], y_train, X_valid[features1], y_valid, pred, price_lambda)

multiple linear regression 1
Mean Squared Error 47,639,437,017.58308
Root Mean Squared Error 218,264.60322
Root Mean Squared Log Error 0.35606
R squared training 0.552
R squared testing 0.501
Adjusted-R squared training 0.551
Adjusted-R squared testing 0.501
intercept [3.96037582]
coefficient [[-0.00035035  0.01953801  0.00850982 -0.06343591  0.00938436]]


### Viacnásobná lineárna regresia - part2

In [10]:
features2 = feature_selection2.select_features_SFS(X_train, y_train, linear_model.LinearRegression)
reg=linear_model.LinearRegression()
reg.fit(X_train[features2],y_train)

pred=reg.predict(X_valid[features2])

print('multiple linear regression 2')
metrics2.evaluate(reg, X_train[features2], y_train, X_valid[features2], y_valid, pred, price_lambda)

multiple linear regression 2
Mean Squared Error 34,944,367,428.43664
Root Mean Squared Error 186,934.12591
Root Mean Squared Log Error 0.25884
R squared training 0.781
R squared testing 0.744
Adjusted-R squared training 0.781
Adjusted-R squared testing 0.744
intercept [1.48311087]
coefficient [[ 1.51482623e-02  3.41215946e-03  3.78452993e-03  2.31928488e-03
   7.04038947e-03 -1.43355076e-04 -2.82001549e-03  5.83690344e-02
  -2.81136532e-02 -1.12521301e+01]]


#### k-násobná krížová validácia pri viacnásobnej lineárnej regresii

In [11]:
reg=linear_model.LinearRegression()
k = 5

X_try = pd.concat([X_train[features2], X_valid[features2]])
y_try = np.concatenate((np.array(y_train).reshape(-1,1), np.array(y_valid).reshape(-1,1)), axis=0)

scores = cross_val_score(reg,  X_try[features2], np.array(y_try).reshape(-1,1), cv=k)
print("Cross-validated scores:", scores.mean())
k = 10
scores = cross_val_score(reg,  X_try[features2], np.array(y_try).reshape(-1,1), cv=k)
print("Cross-validated scores:", scores.mean())

Cross-validated scores: 0.7717217788661574
Cross-validated scores: 0.7718156434452166


### Viacnásobná lineárna regresia - part3

In [12]:
features3 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)
reg=linear_model.LinearRegression()
reg.fit(X_train[features3],y_train)

pred=reg.predict(X_valid[features3])

print('multiple linear regression 3')
metrics2.evaluate(reg, X_train[features3], y_train, X_valid[features3], y_valid, pred, price_lambda)

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436
multiple linear regression 3
Mean Squared Error 32,486,651,893.39338
Root Mean Squared Error 180,240.53898
Root Mean Squared Log Error 0.25336
R squared training 0.793
R squared testing 0.754
Adjusted-R squared training 0.793
Adjusted-R squared testing 0.753
intercept [0.41032586]
coefficient [[-1.25747645e-03  1.91255673e-03  1.36418094e-02 -9.06972876e-03
   2.82037240e-03  9.64036987e-03  2.72934373e-03  2.28954703e-03
   6.11502524e-03 -1.41317852e-04  2.31738952e-03 -2.60579557e-03
   9.45836431e-04 -1.05520017e-03 -9.45611848e-04  1.39617770e-03
   4.85389928e-04  6.01748108e-02 -7.75492425e-03  7.45740712e-03
  -1.89647156e-02 -1.07826081e+01]]


### Polynomiálna regresia 2. stupňa 

In [14]:
features1 = feature_selection2.select_features_SFS(X_train, y_train, linear_model.LinearRegression)

polyfeat=PolynomialFeatures(degree=2)
xtrain_poly=polyfeat.fit_transform(X_train[features1])
xvalid_poly=polyfeat.fit_transform(X_valid[features1])

np_y_train = np.array(y_train).reshape(-1,1)
np_y_valid = np.array(y_valid).reshape(-1,1)

poly=linear_model.LinearRegression()
poly.fit(xtrain_poly,np_y_train)
polypred=poly.predict(xvalid_poly)

print('Polynomial regresion - grade 2')

mean_squared_error=metrics.mean_squared_error(inv_boxcox(np_y_valid, price_lambda),inv_boxcox(polypred, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(np_y_valid, price_lambda),inv_boxcox(polypred, price_lambda))

r2_train = poly.score(xtrain_poly, np_y_train)
r2_test = poly.score(xvalid_poly, np_y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train[features1].shape[0] - 1)) / (X_train[features1].shape[0] - X_train[features1].shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid[features1].shape[0] - 1)) / (X_valid[features1].shape[0] - X_valid[features1].shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 5))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),5))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),5))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Polynomial regresion - grade 2
Mean Squared Error 25599854129.10427
Root Mean Squared Error 159999.54415
Root Mean Squared Log Error 0.23187
R squared training 0.828
R squared testing 0.789
Adjusted-R squared training 0.828
Adjusted-R squared testing 0.789


### Polynomiálna regresia 3. stupňa

In [15]:
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)

polyfeat=PolynomialFeatures(degree=3)
xtrain_poly=polyfeat.fit_transform(X_train[features1])
xvalid_poly=polyfeat.fit_transform(X_valid[features1])
np_y_train = np.array(y_train).reshape(-1,1)
np_y_valid = np.array(y_valid).reshape(-1,1)

poly=linear_model.LinearRegression()
poly.fit(xtrain_poly,np_y_train)
polypred=poly.predict(xvalid_poly)

print('Polynomial regresion - grade 3')
mean_squared_error=metrics.mean_squared_error(inv_boxcox(np_y_valid, price_lambda),inv_boxcox(polypred, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(np_y_valid, price_lambda),inv_boxcox(polypred, price_lambda))

r2_train = poly.score(xtrain_poly, np_y_train)
r2_test = poly.score(xvalid_poly, np_y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train[features1].shape[0] - 1)) / (X_train[features1].shape[0] - X_train[features1].shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid[features1].shape[0] - 1)) / (X_valid[features1].shape[0] - X_valid[features1].shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 5))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),5))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),5))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436
Polynomial regresion - grade 3
Mean Squared Error 236372347755.9625
Root Mean Squared Error 486181.39388
Root Mean Squared Log Error 0.21942
R squared training 0.916
R squared testing 0.828
Adjusted-R squared training 0.916
Adjusted-R squared testing 0.827


In [16]:
features1

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'yr_built',
 'zipcode_1',
 'zipcode_2',
 'zipcode_3',
 'zipcode_4',
 'zipcode_5',
 'zipcode_6',
 'zipcode_7',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 'price_per_sqft']

### Regresný rozhodovací strom

In [17]:
regr_1 = DecisionTreeRegressor(max_depth=10)
regr_1.fit(X_train, y_train)

y_1 = regr_1.predict(X_valid)
mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_1, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_1, price_lambda))

r2_train = regr_1.score(X_train, y_train)
r2_test = regr_1.score(X_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train.shape[0] - 1)) / (X_train.shape[0] - X_train.shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid.shape[0] - 1)) / (X_valid.shape[0] - X_valid.shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 2))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),2))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),2))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Mean Squared Error 23251941137.46
Root Mean Squared Error 152485.87
Root Mean Squared Log Error 0.22
R squared training 0.902
R squared testing 0.803
Adjusted-R squared training 0.902
Adjusted-R squared testing 0.801


### Regresný rozhodovací strom - časť 2

In [18]:
regr_1 = DecisionTreeRegressor(max_depth=8)
regr_1.fit(X_train, y_train)

y_1 = regr_1.predict(X_valid)
mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_1, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_1, price_lambda))

r2_train = regr_1.score(X_train, y_train)
r2_test = regr_1.score(X_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train.shape[0] - 1)) / (X_train.shape[0] - X_train.shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid.shape[0] - 1)) / (X_valid.shape[0] - X_valid.shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 2))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),2))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),2))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Mean Squared Error 23395993447.5
Root Mean Squared Error 152957.49
Root Mean Squared Log Error 0.23
R squared training 0.861
R squared testing 0.802
Adjusted-R squared training 0.861
Adjusted-R squared testing 0.801


### Regresný rozhodovací strom  + GridSearch

In [44]:
def objective_DTree(hyperparameters, iteration):
    regr_1 = DecisionTreeRegressor(**hyperparameters)
    regr_1.fit(X_train, y_train)

    y_1 = regr_1.predict(X_valid)
    mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_1, price_lambda))
    rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_1, price_lambda))

    r2_train = regr_1.score(X_train, y_train)
    r2_test = regr_1.score(X_valid, y_valid)
    adj_r2_train = 1 - (((1 - r2_train) * (X_train.shape[0] - 1)) / (X_train.shape[0] - X_train.shape[1] - 1))
    adj_r2_test = 1 - (((1 - r2_test) * (X_valid.shape[0] - 1)) / (X_valid.shape[0] - X_valid.shape[1] - 1))
    
    score = {
        "MSE": round(mean_squared_error, 2),
        "RMSE": round(np.sqrt(mean_squared_error),2),
        "RMSLE": round(np.sqrt(rmlse),2),
        "R2_train": round(r2_train,3),
        "R2_test": round(r2_test,3),
        "Adj_R2_train": round(adj_r2_train,3),
        "Adj_R2_test": round(adj_r2_test,3)
    }
    
    return [score, hyperparameters]
    

def grid_search(param_grid, max_iterations):
    
    df_results = pd.DataFrame(columns = ['MSE', 'RMSE', 'RMSLE', 'R2_train', 'R2_test', 'Adj_R2_train', 'Adj_R2_test', 'params'],
                                  index = list(range(max_iterations)))
    
    keys, values = zip(*param_grid.items())

    for i, v in enumerate(itertools.product(*values)):
        hyperparameters = dict(zip(keys, v))
        eval_results = objective_DTree(hyperparameters, i)
        
        for k,v in eval_results[0].items():
            df_results[k][i] = v

        df_results['params'][i] = eval_results[1]
        
        if i >= max_iterations-1:
            break
       
    # Sort with best score on top
    df_results.sort_values('R2_test', ascending = False, inplace = True)
    df_results.reset_index(inplace=True)
    
    return df_results

param_grid = {
    'criterion': ['mse', 'friedman_mse', 'mae'],
    'max_depth': np.arange(5,15,1),
    'max_features': np.arange(5,15,1),
    'max_leaf_nodes': [None],
    'min_impurity_decrease': [0.0],
    'min_impurity_split': [None],
    'min_samples_leaf': [1],
    'min_samples_split': [2],
    'min_weight_fraction_leaf': [0.0],
    'splitter': ['best']
}

df_results1 = grid_search(param_grid, 363)
df_results1

,index,MSE,RMSE,RMSLE,R2_train,R2_test,Adj_R2_train,Adj_R2_test,params
0,249,24619862976.84000,156907.18000,0.22000,0.86300,0.80400,0.86300,0.80300,"{'criterion': 'mae', 'max_depth': 9, 'max_feat..."
1,259,24707964196.74000,157187.67000,0.23000,0.88500,0.80100,0.88500,0.80000,"{'criterion': 'mae', 'max_depth': 10, 'max_fea..."
2,67,22264487394.05000,149212.89000,0.23000,0.90700,0.79800,0.90700,0.79700,"{'criterion': 'mse', 'max_depth': 11, 'max_fea..."
3,59,24181808003.74000,155505.01000,0.23000,0.89400,0.79700,0.89400,0.79500,"{'criterion': 'mse', 'max_depth': 10, 'max_fea..."
4,57,23807155327.27000,154295.68000,0.23000,0.88900,0.79700,0.88900,0.79600,"{'criterion': 'mse', 'max_depth': 10, 'max_fea..."
5,56,23241258852.06000,152450.84000,0.23000,0.88900,0.79700,0.88900,0.79600,"{'criterion': 'mse', 'max_depth': 10, 'max_fea..."
6,47,24559807534.74000,156715.69000,0.23000,0.86800,0.79500,0.86800,0.79400,"{'criterion': 'mse', 'max_depth': 9, 'max_feat..."
7,48,23611528899.24000,153660.43000,0.23000,0.87400,0.79500,0.87400,0.79400,"{'criterion': 'mse', 'max_depth': 9, 'max_feat..."
8,69,23361021331.39000,152843.13000,0.23000,0.90500,0.79500,0.90500,0.79400,"{'criterion': 'mse', 'max_depth': 11, 'max_fea..."
9,169,24018711683.42000,154979.71000,0.23000,0.91200,0.79500,0.91100,0.79400,"{'criterion': 'friedman_mse', 'max_depth': 11,..."


In [30]:
np.random.uniform(5,15,1)

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [28]:
model = DecisionTreeRegressor()
model.get_params()

{'criterion': 'mse',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': None,
 'splitter': 'best'}

### Náhodný les

In [19]:
reg_rf = RandomForestRegressor(n_estimators=100, max_depth=8)
reg_rf.fit(X_train, y_train)

y_hat = reg_rf.predict(X_valid)
mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_hat, price_lambda))

r2_train = reg_rf.score(X_train, y_train)
r2_test = reg_rf.score(X_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train.shape[0] - 1)) / (X_train.shape[0] - X_train.shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid.shape[0] - 1)) / (X_valid.shape[0] - X_valid.shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 2))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),2))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),2))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Mean Squared Error 19878363316.99
Root Mean Squared Error 140990.65
Root Mean Squared Log Error 0.2
R squared training 0.892
R squared testing 0.841
Adjusted-R squared training 0.892
Adjusted-R squared testing 0.84


In [46]:
def objective_RF(hyperparameters, iteration):
    regr_1 = RandomForestRegressor(**hyperparameters)
    regr_1.fit(X_train, y_train)

    y_1 = regr_1.predict(X_valid)
    mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_1, price_lambda))
    rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_1, price_lambda))

    r2_train = regr_1.score(X_train, y_train)
    r2_test = regr_1.score(X_valid, y_valid)
    adj_r2_train = 1 - (((1 - r2_train) * (X_train.shape[0] - 1)) / (X_train.shape[0] - X_train.shape[1] - 1))
    adj_r2_test = 1 - (((1 - r2_test) * (X_valid.shape[0] - 1)) / (X_valid.shape[0] - X_valid.shape[1] - 1))
    
    score = {
        "MSE": round(mean_squared_error, 2),
        "RMSE": round(np.sqrt(mean_squared_error),2),
        "RMSLE": round(np.sqrt(rmlse),2),
        "R2_train": round(r2_train,3),
        "R2_test": round(r2_test,3),
        "Adj_R2_train": round(adj_r2_train,3),
        "Adj_R2_test": round(adj_r2_test,3)
    }
    
    return [score, hyperparameters]
    

def grid_search(param_grid, max_iterations):
    
    df_results = pd.DataFrame(columns = ['MSE', 'RMSE', 'RMSLE', 'R2_train', 'R2_test', 'Adj_R2_train', 'Adj_R2_test', 'params'],
                                  index = list(range(max_iterations)))
    
    keys, values = zip(*param_grid.items())

    for i, v in enumerate(itertools.product(*values)):
        hyperparameters = dict(zip(keys, v))
        eval_results = objective_RF(hyperparameters, i)
        
        for k,v in eval_results[0].items():
            df_results[k][i] = v

        df_results['params'][i] = eval_results[1]
        
        if i >= max_iterations-1:
            break
       
    # Sort with best score on top
    df_results.sort_values('R2_test', ascending = False, inplace = True)
    df_results.reset_index(inplace=True)
    
    return df_results

param_grid = {
#     'bootstrap': True,
#     'criterion': 'mse',
    'max_depth': np.arange(5,15,1),
#     'max_features': 'auto',
#     'max_leaf_nodes': None,
#     'min_impurity_decrease': 0.0,
#     'min_impurity_split': None,
#     'min_samples_leaf': 1,
#     'min_samples_split': 2,
#     'min_weight_fraction_leaf': 0.0,
    'n_estimators': [50,200,700,1000],
#     'n_jobs': None,
#     'oob_score': False,
#     'random_state': None,
#     'verbose': 0,
#     'warm_start': False
}

df_results2 = grid_search(param_grid, 44)
df_results2

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWar

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWar

,index,MSE,RMSE,RMSLE,R2_train,R2_test,Adj_R2_train,Adj_R2_test,params
0,39,16270245415.90000,127554.87000,0.19000,0.96800,0.86000,0.96800,0.85900,"{'max_depth': 14, 'n_estimators': 1000}"
1,35,16372193078.80000,127953.87000,0.19000,0.96100,0.86000,0.96100,0.85900,"{'max_depth': 13, 'n_estimators': 1000}"
2,34,16435278318.09000,128200.15000,0.19000,0.96100,0.85900,0.96100,0.85900,"{'max_depth': 13, 'n_estimators': 700}"
3,29,16608879435.70000,128875.44000,0.19000,0.95200,0.85900,0.95200,0.85800,"{'max_depth': 12, 'n_estimators': 200}"
4,30,16620774170.06000,128921.58000,0.19000,0.95200,0.85900,0.95200,0.85800,"{'max_depth': 12, 'n_estimators': 700}"
5,33,16450329047.23000,128258.84000,0.19000,0.96000,0.85900,0.96000,0.85800,"{'max_depth': 13, 'n_estimators': 200}"
6,31,16661644032.04000,129079.99000,0.19000,0.95200,0.85900,0.95200,0.85800,"{'max_depth': 12, 'n_estimators': 1000}"
7,37,16444684142.90000,128236.83000,0.19000,0.96800,0.85900,0.96700,0.85800,"{'max_depth': 14, 'n_estimators': 200}"
8,38,16264469431.76000,127532.23000,0.19000,0.96800,0.85900,0.96800,0.85900,"{'max_depth': 14, 'n_estimators': 700}"
9,36,16531508588.66000,128574.91000,0.19000,0.96700,0.85800,0.96700,0.85800,"{'max_depth': 14, 'n_estimators': 50}"


In [45]:
RandomForestRegressor().get_params()

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

### LinearSVR

In [21]:
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)

clf = LinearSVR(tol=1e-4, loss='squared_epsilon_insensitive')
X_my_train = X_train[features1]#.values.reshape(-1,1)
X_my_valid = X_valid[features1]#.values.reshape(-1,1)

clf.fit(X_my_train, y_train)

y_hat = clf.predict(X_my_valid)
mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_hat, price_lambda))

r2_train = clf.score(X_my_train, y_train)
r2_test = clf.score(X_my_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_my_train.shape[0] - 1)) / (X_my_train.shape[0] - X_my_train.shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_my_valid.shape[0] - 1)) / (X_my_valid.shape[0] - X_my_valid.shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 2))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),2))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),2))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Squared Error 46836652678.26
Root Mean Squared Error 216417.77
Root Mean Squared Log Error 0.32
R squared training 0.659
R squared testing 0.6
Adjusted-R squared training 0.659
Adjusted-R squared testing 0.598


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### RandomSearch pre LinearSVR

In [24]:
def objective_RS_LinearSVR(hyperparameters, iteration, features1):

    clf = LinearSVR(**hyperparameters)
    X_my_train = X_train[features1]#.values.reshape(-1,1)
    X_my_valid = X_valid[features1]#.values.reshape(-1,1)

    clf.fit(X_my_train, np.ravel(y_train))

    y_hat = clf.predict(X_my_valid)
    mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))
    rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_hat, price_lambda))

    r2_train = clf.score(X_my_train, y_train)
    r2_test = clf.score(X_my_valid, y_valid)
    adj_r2_train = 1 - (((1 - r2_train) * (X_my_train.shape[0] - 1)) / (X_my_train.shape[0] - X_my_train.shape[1] - 1))
    adj_r2_test = 1 - (((1 - r2_test) * (X_my_valid.shape[0] - 1)) / (X_my_valid.shape[0] - X_my_valid.shape[1] - 1))
    
    score = {
        "MSE": round(mean_squared_error, 2),
        "RMSE": round(np.sqrt(mean_squared_error),2),
        "RMSLE": round(np.sqrt(rmlse),2),
        "R2_train": round(r2_train,3),
        "R2_test": round(r2_test,3),
        "Adj_R2_train": round(adj_r2_train,3),
        "Adj_R2_test": round(adj_r2_test,3)
    }
    
    return [score, hyperparameters]

def random_search(param_grid, max_iterations, features):
    
    df_results = pd.DataFrame(columns = ['MSE', 'RMSE', 'RMSLE', 'R2_train', 'R2_test', 'Adj_R2_train', 'Adj_R2_test', 'params'],
                                  index = list(range(max_iterations)))
    
    for i in range(max_iterations):
        hyperparameters = {k: random.choice(v) for k, v in param_grid.items()}
        
        eval_results = objective_RS_LinearSVR(hyperparameters, i, features)
        for k,v in eval_results[0].items():
            df_results[k][i] = v
        df_results['params'][i] = eval_results[1]
    
    # Sort with best score on top
    df_results.sort_values('R2_test', ascending = False, inplace = True)
    df_results.reset_index(inplace=True)
    
    return df_results

param_grid = {
#      "C": np.random.uniform(1.2, 2.0, 100),
#     "dual":[True],
#     "epsilon":np.random.uniform(0.1, 0.2, 100),
#     "fit_intercept":[True],
#     "intercept_scaling":[1],
    "loss":['epsilon_insensitive', 'squared_epsilon_insensitive'],
    "max_iter":[10000],
    "tol":np.random.uniform(1e-6, 5e-4, 100),
#     "verbose":[0]
}

random.seed(50)
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)
df_results = random_search(param_grid, 50, features1)
df_results

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", 

,index,MSE,RMSE,RMSLE,R2_train,R2_test,Adj_R2_train,Adj_R2_test,params
0,41,37849023054.86000,194548.25000,0.27000,0.74100,0.70900,0.74100,0.70700,"{'loss': 'squared_epsilon_insensitive', 'max_i..."
1,22,38103770055.50000,195201.87000,0.27000,0.73300,0.70800,0.73300,0.70700,"{'loss': 'squared_epsilon_insensitive', 'max_i..."
2,1,38390055628.08000,195933.80000,0.27000,0.72900,0.70700,0.72800,0.70600,"{'loss': 'epsilon_insensitive', 'max_iter': 10..."
3,5,38763827416.23000,196885.32000,0.27000,0.72400,0.70500,0.72300,0.70300,"{'loss': 'epsilon_insensitive', 'max_iter': 10..."
4,19,38672124986.97000,196652.29000,0.27000,0.72500,0.70500,0.72400,0.70400,"{'loss': 'epsilon_insensitive', 'max_iter': 10..."
5,49,39115459580.92000,197776.29000,0.28000,0.71900,0.70300,0.71800,0.70100,"{'loss': 'squared_epsilon_insensitive', 'max_i..."
6,39,38149264578.57000,195318.37000,0.28000,0.74500,0.70300,0.74500,0.70100,"{'loss': 'squared_epsilon_insensitive', 'max_i..."
7,33,39318306021.67000,198288.44000,0.28000,0.71700,0.70200,0.71600,0.70000,"{'loss': 'epsilon_insensitive', 'max_iter': 10..."
8,28,39381002019.73000,198446.47000,0.28000,0.71600,0.70100,0.71500,0.70000,"{'loss': 'squared_epsilon_insensitive', 'max_i..."
9,40,38311119499.05000,195732.26000,0.28000,0.74500,0.70100,0.74500,0.69900,"{'loss': 'squared_epsilon_insensitive', 'max_i..."


### SGD + Nystroem

In [22]:
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)

clf = SGDRegressor(tol=1e-3, penalty='l1', shuffle=False)
feature_map_nystroem = Nystroem(gamma=.2, random_state=1, n_components=300)

data_transformed_train = feature_map_nystroem.fit_transform(X_train[features1])
data_transformed_valid = feature_map_nystroem.transform(X_valid[features1])

clf.fit(data_transformed_train, np.ravel(y_train))
y_hat = clf.predict(data_transformed_valid)


mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_hat, price_lambda))

r2_train = clf.score(data_transformed_train, y_train)
r2_test = clf.score(data_transformed_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train.shape[0] - 1)) / (X_train.shape[0] - X_train[features1].shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid.shape[0] - 1)) / (X_valid.shape[0] - X_valid[features1].shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 2))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),2))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),2))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436
Mean Squared Error 120493650338.01
Root Mean Squared Error 347121.95
Root Mean Squared Log Error 0.56
R squared training -0.221
R squared testing -0.192
Adjusted-R squared training -0.223
Adjusted-R squared testing -0.198


### SGD + RandomSearch

In [23]:
def objective_RS_SGD(hyperparameters, iteration, features1):

    clf = SGDRegressor(**hyperparameters)
    feature_map_nystroem = Nystroem(gamma=.2, random_state=1, n_components=300)

    data_transformed_train = feature_map_nystroem.fit_transform(X_train[features1])
    data_transformed_valid = feature_map_nystroem.transform(X_valid[features1])

    clf.fit(data_transformed_train, np.ravel(y_train))
    y_hat = clf.predict(data_transformed_valid)
    
    mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))
    rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_hat, price_lambda))

    r2_train = clf.score(X_my_train, y_train)
    r2_test = clf.score(X_my_valid, y_valid)
    adj_r2_train = 1 - (((1 - r2_train) * (X_my_train.shape[0] - 1)) / (X_my_train.shape[0] - X_my_train.shape[1] - 1))
    adj_r2_test = 1 - (((1 - r2_test) * (X_my_valid.shape[0] - 1)) / (X_my_valid.shape[0] - X_my_valid.shape[1] - 1))
    
    score = {
        "MSE": round(mean_squared_error, 2),
        "RMSE": round(np.sqrt(mean_squared_error),2),
        "RMSLE": round(np.sqrt(rmlse),2),
        "R2_train": round(r2_train,3),
        "R2_test": round(r2_test,3),
        "Adj_R2_train": round(adj_r2_train,3),
        "Adj_R2_test": round(adj_r2_test,3)
    }
    
    return [score, hyperparameters]


def random_search(param_grid, max_iterations, features):
    
    df_results = pd.DataFrame(columns = ['MSE', 'RMSE', 'RMSLE', 'R2_train', 'R2_test', 'Adj_R2_train', 'Adj_R2_test', 'params'],
                                  index = list(range(max_iterations)))
    
    for i in range(max_iterations):
        hyperparameters = {k: random.choice(v) for k, v in param_grid.items()}
        
        eval_results = objective(hyperparameters, i, features)
        for k,v in eval_results[0].items():
            df_results[k][i] = v
        df_results['params'][i] = eval_results[1]
    
    # Sort with best score on top
    df_results.sort_values('R2_test', ascending = False, inplace = True)
    df_results.reset_index(inplace=True)
    
    return df_results

param_grid = {
    'alpha': np.random.uniform(1e-5,1e-3,100),
#     'average': [True,False],
    'early_stopping': [True],
#     'epsilon': [0.1],
#     'eta0': np.random.uniform(0.001,0.1,100),
#     'fit_intercept': [True],
    'l1_ratio': np.random.uniform(0.1,0.9,100),
    'learning_rate': ['constant', 'adaptive','invscaling'],
    'loss': ['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
    'max_iter': [10000],
    'n_iter_no_change': [5],
    'penalty': ['l1','l2','elasticnet'],
    'power_t': np.random.uniform(0.15,0.4,100),
    'shuffle': [False],
    "tol":np.random.uniform(1e-4, 1e-2, 100),
    'validation_fraction': np.arange(0.1, 0.3, 0.05),
}

random.seed(50)
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)
df_results2 = random_search(param_grid, 50, features1)
df_results2

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436
{'alpha': 0.0006442253346602404, 'early_stopping': True, 'l1_ratio': 0.3540958838215098, 'learning_rate': 'invscaling', 'loss': 'huber', 'max_iter': 10000, 'n_iter_no_change': 5, 'penalty': 'l1', 'power_t': 0.39473062346249244, 'shuffle': False, 'tol': 0.008125160287660194, 'validation_fraction': 0.1}
[]
{'alpha': 0.0005919473455452715, 'early_stopping': True, 'l1_ratio': 0.8084613155735152, 'learning_rate': 'adaptive', 'loss': 'epsilon_insensitive', 'max_iter': 10000, 'n_iter_no_change': 5, 'penalty': 'l1', 'power_t': 0.19059051387349515, 'shuffle': False, 'tol': 0.005757411767652302, 'validation_fraction': 0.1}
[]
{'alpha': 0.000770741951165923, 'early_stopping': True, 'l1_ratio': 0.5772391997537714, 'learning_rate': 'invscaling', 'loss': 'squared_epsilon_insensitive', 'max_iter': 10000, 'n_iter_no_change': 5, 'penalty': 'l2', 'power_t': 0.18123570218028312, 'shuffle': False, 'tol': 0.00117332804701976

{'alpha': 0.0003507154089041436, 'early_stopping': True, 'l1_ratio': 0.47115222583747063, 'learning_rate': 'constant', 'loss': 'squared_epsilon_insensitive', 'max_iter': 10000, 'n_iter_no_change': 5, 'penalty': 'elasticnet', 'power_t': 0.3105541325098048, 'shuffle': False, 'tol': 0.003397818813751871, 'validation_fraction': 0.20000000000000004}
[]
{'alpha': 0.0005111695260623213, 'early_stopping': True, 'l1_ratio': 0.7704732102357629, 'learning_rate': 'invscaling', 'loss': 'squared_epsilon_insensitive', 'max_iter': 10000, 'n_iter_no_change': 5, 'penalty': 'elasticnet', 'power_t': 0.19059051387349515, 'shuffle': False, 'tol': 0.0034390404121555033, 'validation_fraction': 0.25000000000000006}
[]
{'alpha': 0.00012104396623842693, 'early_stopping': True, 'l1_ratio': 0.5000268293650408, 'learning_rate': 'adaptive', 'loss': 'huber', 'max_iter': 10000, 'n_iter_no_change': 5, 'penalty': 'elasticnet', 'power_t': 0.30815349678318277, 'shuffle': False, 'tol': 0.009274156020935228, 'validation_fra

,index,MSE,RMSE,RMSLE,R2_train,R2_test,Adj_R2_train,Adj_R2_test,params
0,42,69287017680.61000,263224.27000,0.39000,0.47100,0.42800,0.47000,0.42600,"{'alpha': 2.729279923260973e-05, 'early_stoppi..."
1,11,71476957145.03000,267351.75000,0.39000,0.45000,0.40900,0.44900,0.40600,"{'alpha': 0.00012104396623842693, 'early_stopp..."
2,18,73659834459.28000,271403.45000,0.40000,0.43300,0.39500,0.43300,0.39200,"{'alpha': 0.00045300147183052525, 'early_stopp..."
3,15,82055522381.42000,286453.35000,0.42000,0.38600,0.34700,0.38500,0.34400,"{'alpha': 0.0007269816253602855, 'early_stoppi..."
4,31,83429500712.62000,288841.65000,0.42000,0.35500,0.32700,0.35400,0.32300,"{'alpha': 0.0007613502058801933, 'early_stoppi..."
5,14,87484863602.53000,295778.40000,0.45000,0.26700,0.25000,0.26600,0.24600,"{'alpha': 8.623753065035408e-05, 'early_stoppi..."
6,6,94141452319.59000,306824.79000,0.46000,0.21500,0.20500,0.21400,0.20100,"{'alpha': 0.0007159519994880934, 'early_stoppi..."
7,27,95081875952.86000,308353.49000,0.47000,0.15300,0.14500,0.15100,0.14100,"{'alpha': 0.00012104396623842693, 'early_stopp..."
8,5,102598307496.25000,320309.71000,0.49000,0.09300,0.09500,0.09200,0.09000,"{'alpha': 1.9909508382162684e-05, 'early_stopp..."
9,36,104610599908.67999,323435.62000,0.51000,-0.00000,-0.00000,-0.00100,-0.00500,"{'alpha': 0.0009594637031885341, 'early_stoppi..."


In [14]:
df_results2["params"][3]

{'alpha': 0.00010649479903447461,
 'early_stopping': True,
 'l1_ratio': 0.2730629352844155,
 'learning_rate': 'constant',
 'loss': 'huber',
 'max_iter': 10000,
 'n_iter_no_change': 5,
 'penalty': 'l2',
 'power_t': 0.1764792717720897,
 'shuffle': False,
 'tol': 0.008364349005682366,
 'validation_fraction': 0.20000000000000004}